<a href="https://colab.research.google.com/github/larregui/IST718-Project/blob/main/UFO_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [1]:
import pandas as pd #data frame operations
import numpy as np #arrays and math functions
#from scipy.stats import uniform #for training and test splits
#import statsmodels.api as sm  # statistical models (including regression)
#import statsmodels.formula.api as smf  # R-like model specification
import matplotlib.pyplot as plt #2D plotting
import seaborn as sns #seaborn for plotting
#from sklearn.linear_model import LinearRegression
#import plotly.graph_objs as go
#from plotly import tools
#from plotly.offline import iplot, init_notebook_mode

#Import Datasets

In [19]:
ufo = pd.read_csv("/content/newufo.csv")
wea = pd.read_csv("/content/consolidated_weather_V03.csv")

#Data Cleanup

In [20]:
#change datatype for the following columns
ufo["latitude"] = pd.to_numeric(ufo["latitude"], errors="coerce")
ufo['datetime'] = (pd.to_datetime(ufo['datetime'], format='%Y/%m/%d'))
ufo['date posted'] = (pd.to_datetime(ufo['date posted'], format='%Y/%m/%d'))
#add column day
ufo['day'] = pd.DatetimeIndex(ufo['datetime']).day
#drop unnecessary columns
cols2drop=[ 'Unnamed: 0','duration (hours/min)','date posted']
ufo=ufo.drop(columns=cols2drop)

In [21]:
# Creating seasons column
# Seasons of the ufo appearances
def output_season_from(x):
    
    if x in range(3,6):
        return "Spring"
    if x in range(6,9):
        return "Summer"
    if x in range(9,12):
        return "Autumn"
    if x == 12 or x == 1 or x == 2:
        return "Winter"

ufo["season"] = ufo['datetime'].apply(output_season_from)

In [22]:
wea['dateInt']=wea['year'].astype(str) + wea['month'].astype(str).str.zfill(2)+ wea['mday'].astype(str).str.zfill(2)
wea['datetime'] = pd.to_datetime(wea['dateInt'], format='%Y%m%d')
wea['state']=wea['state'].str.lower()
wea['city']=wea['city'].str.lower()
wea['shape']=wea['shape'].str.lower()
cols2drop=[ 'colour','month','pressure','heatindex','windchill', 'precip', 
            'wgust', 'wdire','dewpt','wspd','hum','conds', 'mday','year','hour','id','dateInt']

wea=wea.drop(columns=cols2drop)

# Shows all the null values
#print(wea.isnull().sum())

wea=wea.dropna()
wea.shape

(50963, 12)

#Merge datatsets

In [23]:
df = pd.merge(ufo, wea, on=["city", "state", "datetime", "shape"])
df.describe()

,duration (seconds),latitude,longitude,year,month,military_base_dist,airport_dist,day,temp,hail,rain,vis,thunder,fog,tornado,snow
count,3.535600e+04,35356.000000,35356.000000,35356.000000,35356.000000,35356.000000,35356.000000,35356.000000,35356.000000,35356.0,35356.000000,35356.000000,35356.000000,35356.000000,35356.000000,35356.000000
mean,3.640002e+03,38.383515,-95.421423,2007.543896,6.889043,5000.481879,3.779888,15.307642,-0.040161,0.0,0.021976,-0.003462,0.005968,0.004101,0.000028,0.004299
std,2.908908e+05,5.596983,18.223826,4.605761,3.327509,1132.724850,3.878917,8.871921,1.006675,0.0,0.146609,1.054241,0.077022,0.063910,0.005318,0.065428
min,1.000000e-02,19.063889,-170.478889,1997.000000,1.000000,1579.967092,0.022381,1.000000,-5.419079,0.0,0.000000,-2.740390,0.000000,0.000000,0.000000,0.000000
25%,3.000000e+01,34.134722,-114.888542,2004.000000,4.000000,3872.748270,1.231377,8.000000,-0.699044,0.0,0.000000,0.064273,0.000000,0.000000,0.000000,0.000000
50%,1.800000e+02,38.998889,-89.213611,2008.000000,7.000000,5314.815203,2.740357,15.000000,0.077412,0.0,0.000000,0.064273,0.000000,0.000000,0.000000,0.000000
75%,6.000000e+02,42.011111,-80.699722,2012.000000,10.000000,5868.691832,4.821439,23.000000,0.731270,0.0,0.000000,0.064273,0.000000,0.000000,0.000000,0.000000
max,5.262320e+07,70.636944,-66.984722,2014.000000,12.000000,7238.415091,56.585655,31.000000,4.102724,0.0,1.000000,30.898139,1.000000,1.000000,1.000000,1.000000


#Classifier

In [26]:
df.loc[df['tornado'] ==1, 'Credible'] = 'NO'
df.loc[df['fog'] ==1, 'Credible'] = 'NO'
df.loc[df['snow'] ==1, 'Credible'] = 'NO'
df.loc[df['rain'] ==1, 'Credible'] = 'NO'
df.loc[df['thunder'] ==1, 'Credible'] = 'NO'
df.loc[df['hail'] ==1, 'Credible'] = 'NO'
#df.loc[df['vis'] >= 0.064273, 'Credible'] = 'NO'
df.loc[df['airport_dist'] <= 3, 'Credible'] = 'NO'
df.loc[df['military_base_dist'] <= 3, 'Credible'] = 'NO'

In [27]:
df.head()

,datetime,city,state,country,shape,duration (seconds),comments,latitude,longitude,time,year,month,coord,military_base_name,military_base_dist,airport_name,airport_dist,day,season,temp,hail,rain,vis,thunder,fog,tornado,snow,Credible
0,1997-10-10,mesa,az,us,sphere,30.0,A small dark purple quad-thruster craft hoveri...,33.422222,-111.821944,17:00,1997,10,"('33.4222222', -111.8219444)",Eareckson AS,3454.409479,Mesa Hospital Medical Center Heliport,0.311086,10,None,1.660974,0,0,0.064273,0,0,0,0,NO
1,1997-10-10,bonaire,ga,us,triangle,300.0,Triangular Object Sighted at Very Close Range,32.543611,-83.596111,20:00,1997,10,"('32.5436111', -83.59611109999999)",Eareckson AS,5404.265953,Warner Robins Air Park,7.589948,10,None,0.629105,0,0,0.900445,0,0,0,0,NO
2,1998-10-10,las vegas,nv,us,cigar,600.0,White&#44 vertical cigar shape floating around...,36.175000,-115.136389,17:30,1998,10,"('36.1750000', -115.1363889)",Eareckson AS,3589.816966,City Hall Complex Heliport,0.518080,10,None,1.242096,0,0,2.868935,0,0,0,0,NO
3,1998-10-10,mooresville,nc,us,light,2.0,Star gazing in back yard with telescope and bi...,35.584722,-80.810278,20:50,1998,10,"('35.5847222', -80.8102778)",Eareckson AS,5775.016814,Hunter Construction Heliport,3.038387,10,None,0.016113,0,0,0.064273,0,0,0,0,NO
4,1998-10-10,delano,tn,us,fireball,15.0,I came home from work one night and I seen a b...,35.265000,-84.553333,23:50,1998,10,"('35.2650000', -84.55333329999999)",Eareckson AS,5452.721410,Chilhowee Gliderport,5.153607,10,None,-0.096269,0,0,0.064273,0,0,0,0,NO
